In [ ]:
import sys
sys.path.append('..')

In [ ]:
import torch.multiprocessing as mp

# Fix multiprocessing trên Windows
if __name__ == '__main__':
    mp.set_start_method('spawn', force=True)

In [ ]:
from modular import data_setup, engine, models, helper
import torch

In [ ]:
# Model setup
vitb16_model, transforms = models.create_vit_model(5, 42)
vit_from_scratch_model = models.create_vit_tiny_cpu(5, 42)

In [ ]:
# Data setup

train_dir = '../data/train'
test_dir = '../data/test'

train_dataloader, test_dataloader, class_names, targets = data_setup.create_dataloaders(train_dir, test_dir, transforms)

In [ ]:
# Setup loss function and optimizer
EPOCHS = 3
LR = 0.001
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
def train_and_evaluate(model, train_dataloader, test_dataloader, loss_fn, epochs, device='cpu', save_path = '../reports/vit_from_scratch_training_results.jpg'):
    optimizer = torch.optim.Adam(model.parameters(), lr=LR) 
    results = engine.train(model=model,
                           train_dataloader=train_dataloader,
                           test_dataloader=test_dataloader,
                           optimizer=optimizer,
                           loss_fn=loss_fn,
                           epochs=epochs,
                           device=device)
    
    fig = helper.plot_results(results, epochs=epochs, save_path=save_path)

    return results

In [ ]:
my_vit_results = train_and_evaluate(vit_from_scratch_model, train_dataloader, test_dataloader, loss_fn, epochs=EPOCHS, device='cpu')

In [ ]:
vit_b16_results = train_and_evaluate(vitb16_model, train_dataloader, test_dataloader, loss_fn, epochs=EPOCHS, device='cpu', save_path='../reports/vit_b16_training_results.jpg')

In [ ]:
fig, ax, my_vit_pred = helper.predict_and_plot_confusion_matrix(test_dataloader, vit_from_scratch_model, class_names, targets, save_path='../reports/vit_from_scratch_confusion_matrix.jpg')

In [ ]:
fig, ax, vit_b16_pred = helper.predict_and_plot_confusion_matrix(test_dataloader, vitb16_model, class_names, targets, save_path='../reports/vit_b16_confusion_matrix.jpg')